In [3]:
from sklearn.datasets import load_files
import os

In [8]:
PATH_TO_DATA = '/Users/mzulliev/education/ods_mlcourse/data/aclImdb'

In [9]:
!du -hs $PATH_TO_DATA

487M	/Users/mzulliev/education/ods_mlcourse/data/aclImdb


In [10]:
!du -hs $PATH_TO_DATA/train
!du -hs $PATH_TO_DATA/test

365M	/Users/mzulliev/education/ods_mlcourse/data/aclImdb/train
121M	/Users/mzulliev/education/ods_mlcourse/data/aclImdb/test


In [20]:
%%time
train_reviews = load_files(os.path.join(PATH_TO_DATA, 'train'))

CPU times: user 483 ms, sys: 1.37 s, total: 1.86 s
Wall time: 7.95 s


In [21]:
%%time
test_reviews = load_files(os.path.join(PATH_TO_DATA, 'test'))

CPU times: user 158 ms, sys: 424 ms, total: 582 ms
Wall time: 2.56 s


In [22]:
len(train_reviews.data)

75000

In [24]:
train_reviews.data[0]

b'Full of (then) unknown actors TSF is a great big cuddly romp of a film.<br /><br />The idea of a bunch of bored teenagers ripping off the local sink factory is odd enough, but add in the black humour that Forsyth & Co are so good at and your in for a real treat.<br /><br />The comatose van driver by itself worth seeing, and the canal side chase is just too real to be anything but funny.<br /><br />And for anyone who lived in Glasgow it\'s a great "Oh I know where that is" film.'

In [25]:
train_reviews.target[0]

2

In [43]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [44]:
a = np.zeros([5, 5])
a[0, 4] = 1
a[4, 1] = 5
a[2, 3] = 3
a[3, 0] = 7
a[3, 4] = 1

In [45]:
a

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 3., 0.],
       [7., 0., 0., 0., 1.],
       [0., 5., 0., 0., 0.]])

In [47]:
pd.DataFrame(a, columns=['apple', 'wax', 'sadness', 'luck', 'girl'])

,apple,wax,sadness,luck,girl
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,3.0,0.0
3,7.0,0.0,0.0,0.0,1.0
4,0.0,5.0,0.0,0.0,0.0


In [48]:
b = csr_matrix(a)

In [49]:
b

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (5, 5)>

In [50]:
b.todense()

matrix([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 3., 0.],
        [7., 0., 0., 0., 1.],
        [0., 5., 0., 0., 0.]])

In [51]:
b.nonzero()

(array([0, 2, 3, 3, 4], dtype=int32), array([4, 3, 0, 4, 1], dtype=int32))

In [52]:
b.data

array([1., 3., 7., 1., 5.])

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
%%time
cv = CountVectorizer()
X_train_sparse = cv.fit_transform(train_reviews.data)

CPU times: user 4.75 s, sys: 166 ms, total: 4.92 s
Wall time: 5.5 s


In [61]:
len(cv.vocabulary_)

124255

In [62]:
X_test_sparse = cv.transform(test_reviews.data)

In [63]:
X_train_sparse.shape, X_test_sparse.shape

((75000, 124255), (25000, 124255))

In [64]:
X_train_sparse

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10359806 stored elements and shape (75000, 124255)>